In [ ]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 946.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
# RAG 1. load doc (done), 2. chunking, splits, 3. embeding - indexing, 4. retrieve

In [ ]:
from langchain_upstage import UpstageLayoutAnalysisLoader

# 1. load doc
layzer = UpstageLayoutAnalysisLoader("/expling-syllabus-240728.pdf", api_key="up_EbdQcRYJS0LhtlVCzTE6iwuimlUiE", output_type="html")
docs = layzer.load()

In [ ]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 21


In [ ]:
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from langchain.docstore.document import Document


In [ ]:
# 3. Embed & indexing
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=UpstageEmbeddings(api_key="up_EbdQcRYJS0LhtlVCzTE6iwuimlUiE", model="solar-embedding-1-large"),
)

In [ ]:
# 4. retrive
retriever = vectorstore.as_retriever()
result_docs = retriever.invoke("When is the office hours for Experimental Linguistics?")
print(len(result_docs))
print(result_docs[0].page_content[:100])

4
<h1 id='0' style='font-size:18px'>Experimental Linguistics<br>Syllabus</h1><br><table id='1' style='


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key="up_EbdQcRYJS0LhtlVCzTE6iwuimlUiE")

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [ ]:
chain.invoke({"question": "When is the office hours for Experimental Linguistics?", "Context": result_docs})

'The office hours for Experimental Linguistics are MonTues 2-3pm.'

In [ ]:
chain.invoke({"question": "What tools will be used in Experimental Linguistics?", "Context": result_docs})

'The tools that will be used in Experimental Linguistics are Praat, OpenSesame or Psychopy, IBEX or PCIbex or JsPsych, and R.'

In [ ]:
chain.invoke({"question": "Can you tell me the required readings for each weeks in Experimental Linguistics?", "Context": result_docs})

'The required readings for each week in Experimental Linguistics are as follows:\n\nWeek 1:\n- Optional reading: Podesva and Sharma (2014) Ch. 2\n- Lab: Using Latex, presenting linguistic data in paper\n\nWeek 2:\n- Required reading: Mike Frank’s textbook: Ch. 1–2, 8–9\n- Optional reading: Podesva and Sharma (2014) Ch. 7, Rosenthal and Rosnow (2008) Ch. 7\n\nWeek 3:\n- Diving deeper into a sample linguistic experiment\n- No specific readings mentioned\n\nPlease note that the information for Week 3 is not present in the context.'

In [ ]:
chain.invoke({"question": "I haven't taken Phonology, Morphology and Syntax. Can I take  Experimental Linguistics?", "Context": result_docs})

'No, you cannot take Experimental Linguistics without taking Phonology, Morphology and Syntax. According to the course requirements, students are strongly encouraged to have taken at least one of these classes as a prerequisite.'

In [ ]:
chain.invoke({"question": "What can I learn through Experimental Linguistics?", "Context": result_docs})

'Through Experimental Linguistics, you can learn how to transform linguistic hypotheses into empirically testable predictions, design an experimental study, implement experiments using various tools, analyze the results, and understand the types of questions and debates in various subfields of linguistics that have motivated experimental approaches.'

In [ ]:
chain.invoke({"question": "Could you please write an email to Instructor requesting attendance recognition for absence on September 17th?", "Context": result_docs})

'Dear Instructor Sunwoo Jeong,\n\nI am writing to request attendance recognition for my absence on September 17th. Unfortunately, I was not able to attend class that day due to a personal matter. I would like to request that my absence be recognized and that I be given the opportunity to make up any missed work or assignments.\n\nThank you for your understanding and cooperation.\n\nSincerely,\n[Your Name]'

In [ ]:
chain.invoke({"question": "Where all readings for Experimental Linguistics are posted?", "Context": result_docs})

'The required and optional readings for Experimental Linguistics are posted in the context. The required reading for Week 2 is Mike Frank’s textbook: Ch. 1–2, 8–9, and the optional reading is Podesva and Sharma (2014) Ch. 7, Rosenthal and Rosnow (2008) Ch. 7.'

In [ ]:
chain.invoke({"question": "When is the lab session for Experimental Linguistics? How many times?", "Context": result_docs})

'The lab session for Experimental Linguistics is conducted during the class time, which is MonWeds 12:30-1:45pm. The number of lab sessions is not explicitly mentioned, but it is implied that there are 4 in-lab assignments throughout the course.'